## Librerias


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import(
    KFold,
    ShuffleSplit,
    cross_val_score,
    learning_curve,
    train_test_split,
)
from sklearn.pipeline import Pipeline

## Carga del dataset (prepocesado)

In [16]:
df=pd.read_csv('datos_personas.csv')

In [17]:
df

,edad,dependientes,salario,categoria_nw_cliente,codigo_sucursal,saldo_actual,saldo_final_mes_anterior,saldo_mes_actual,saldo_mes_anterior,objetivo,genero_Mujer,ocupacion_Independiente,ocupacion_Pensionado,ocupacion_Trabajador
0,66,0,187,2,755,1458,1458,1458,1458,0,0,1,0,0
1,31,0,146,2,41,3913,5815,5006,5070,0,0,0,0,1
2,42,2,1494,3,388,927,1401,1157,1677,1,0,1,0,0
3,42,0,1096,2,1666,15202,16059,15719,15349,0,1,1,0,0
4,72,0,1020,1,1,7006,7714,7076,7755,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22318,85,0,1589,2,389,1741,1741,1741,1741,0,0,0,1,0
22319,10,0,1020,2,1207,1076,1076,1076,1076,0,1,0,0,0
22320,47,0,1096,2,588,65511,61017,61078,57564,1,0,0,0,1
22321,50,3,1219,3,274,1625,1625,1625,1625,0,0,1,0,0


In [18]:
features = ['edad', 'dependientes', 'salario','saldo_actual','genero_Mujer','ocupacion_Independiente','ocupacion_Pensionado','ocupacion_Trabajador']

In [19]:
x_features = df[features]

In [20]:
y_target = df['objetivo']

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=0.25, stratify=y_target)

In [22]:
x_train.shape, y_train.shape

((16742, 8), (16742,))

In [23]:
x_test.shape, y_test.shape

((5581, 8), (5581,))

In [24]:
x_train

,edad,dependientes,salario,saldo_actual,genero_Mujer,ocupacion_Independiente,ocupacion_Pensionado,ocupacion_Trabajador
20248,45,1,1232,706,1,0,0,1
3626,27,0,1322,417,0,1,0,0
4373,30,3,575,1660,0,1,0,0
7740,68,0,1020,4771,0,0,1,0
9506,47,0,665,16624,0,0,0,1
...,...,...,...,...,...,...,...,...
16853,36,0,1111,5904,0,0,0,1
3523,50,2,1149,1029,1,1,0,0
1483,71,1,1096,5,0,0,0,1
18067,43,0,1020,10938,1,1,0,0


In [25]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16742 entries, 20248 to 1096
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   edad                     16742 non-null  int64
 1   dependientes             16742 non-null  int64
 2   salario                  16742 non-null  int64
 3   saldo_actual             16742 non-null  int64
 4   genero_Mujer             16742 non-null  int64
 5   ocupacion_Independiente  16742 non-null  int64
 6   ocupacion_Pensionado     16742 non-null  int64
 7   ocupacion_Trabajador     16742 non-null  int64
dtypes: int64(8)
memory usage: 1.1 MB


In [26]:
y_test

,objetivo
11511,0
20715,0
18962,0
4299,0
2497,0
...,...
12832,0
3978,0
15221,0
1982,0


In [27]:
class HeuristicModel(BaseEstimator, ClassifierMixin):
    """
    Modelo heurístico compatible con scikit-learn.
    Toma decisiones basadas en reglas simples sobre las columnas del dataset.
    """

    def __init__(self, dependientes_threshold=3):
        self.dependientes_threshold = dependientes_threshold

    def fit(self, X, y=None):
        # Guarda las clases únicas para compatibilidad con scikit-learn
        if y is not None:
            self.classes_ = np.unique(y)
        return self

    def predict(self, X):
        predictions = []

        # Iterar sobre las filas del DataFrame
        for _, row in X.iterrows():
            # Regla heurística: si 'dependientes' menores al threshold
            if row['dependientes'] <= self.dependientes_threshold:
                predictions.append(1)
            else:
                predictions.append(0)

        return np.array(predictions)